In [57]:
import pandas as pd
import numpy as np
import cudf

IS_SUBMIT = False
IS_TRAIN = False

if IS_TRAIN:
    train = cudf.read_parquet('/root/autodl-tmp/ottodata/valid/train.parquet')
    test = cudf.read_parquet('/root/autodl-tmp/ottodata/valid/test.parquet')
else:
    train = cudf.read_parquet('/root/autodl-tmp/ottodata/train.parquet')
    test = cudf.read_parquet('/root/autodl-tmp/ottodata/test.parquet')

In [58]:
data = cudf.concat([train,test])

In [59]:
item_label = cudf.read_parquet('/root/autodl-tmp/ottodata/tmp/item_label3.parquet')

In [60]:
test = test.merge(item_label,on='aid',how='left')
data = data.merge(item_label,on='aid',how='left')

In [61]:
test = test.sort_values(['session','ts'])
data = data.sort_values(['session','ts'])


In [62]:
popular_items = data.groupby(['item_label','type','aid'])['ts'].count().reset_index()

In [63]:
popular_items = popular_items.sort_values(by=['item_label','type','ts'], ascending=False)

In [64]:
popular_items['n'] = popular_items.groupby(['item_label','type'])['aid'].cumcount()
popular_items = popular_items[popular_items['n'] < 20]
# popular_items = popular_items.to_pandas().groupby('type')['aid'].apply(list).reset_index()

In [65]:
popular_items = popular_items.groupby(['item_label','type'])['aid'].collect().reset_index()

In [66]:
most_label = test.groupby(['session','item_label'])['ts'].count().reset_index()
most_label = most_label.sort_values(['session','ts'],ascending=[True,False])
most_label = most_label.groupby('session')['item_label'].first()

In [67]:
most_label = most_label.reset_index()

In [68]:
id2type = {0: 'clicks', 1: 'carts', 2: 'orders'}
popular_items['type'] = popular_items['type'].map(id2type)

In [69]:
pred_dfs = []
for t in  ['clicks', 'carts', 'orders']:
    pred_df = most_label.copy()
    label =  popular_items[popular_items['type'] == t]
    pred_df = pred_df.merge(label, on='item_label', how='left')
    pred_df['type'] = t
    pred_df =pred_df.drop(columns = ['item_label'])
    pred_df = pred_df.sort_values(['session'])
    pred_dfs.append(pred_df)
pred_df = cudf.concat(pred_dfs)

In [70]:
pred_df.columns = ['session','type','labels']
pred_df = pred_df.to_pandas()
pred_df = pred_df.reset_index(drop=True)
pred_df['labels'] = pred_df['labels'].apply(lambda d: d if isinstance(d,np.ndarray) else np.array([]))
pred_df['labels'] = pred_df['labels'].apply(lambda d: list(d))

In [71]:
test_labels = pd.read_parquet('/root/autodl-tmp/ottodata/valid/test_labels.parquet')
score = 0
weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
for t in ['clicks','carts','orders']:
    sub = pred_df.loc[pred_df['type']==t].copy()
    test_labels_sub = test_labels.loc[test_labels['type']==t]
    test_labels_sub = test_labels_sub.merge(sub, how='left', on=['session'])
    test_labels_sub['hits'] = test_labels_sub.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels[:20]))), axis=1)
    test_labels_sub['gt_count'] = test_labels_sub.ground_truth.str.len().clip(0,20)
    recall = test_labels_sub['hits'].sum() / test_labels_sub['gt_count'].sum()
    score += weights[t]*recall
    print(f'{t} recall =',recall)
print('=============')
print('Overall Recall =',score)
print('=============')

TypeError: 'float' object is not subscriptable

In [72]:
if IS_TRAIN:
    pred_df.to_parquet('/root/autodl-tmp/ottodata/tmp/hot4train.parquet')
else:
    pred_df.to_parquet('/root/autodl-tmp/ottodata/tmp/hot4test.parquet')

In [73]:
pred_df

,session,type,labels
0,12899779,clicks,"[610887, 1387440, 1806664, 1634357, 635921, 12..."
1,12899780,clicks,"[1502122, 1586171, 636101, 1142000, 1419849, 8..."
2,12899781,clicks,"[1460571, 620545, 811371, 102345, 670006, 3317..."
3,12899782,clicks,"[889671, 498405, 552213, 424203, 905448, 60838..."
4,12899783,clicks,"[187161, 232749, 1793171, 155991, 838646, 1502..."
...,...,...,...
5009614,14571577,orders,"[399315, 1308823, 573273, 195833, 1405280, 176..."
5009615,14571578,orders,"[109382, 141364, 386723, 1109341, 349161, 1018..."
5009616,14571579,orders,"[1767620, 1829825, 945766, 1634188, 1106245, 1..."
5009617,14571580,orders,"[773782, 1265851, 32322, 484711, 672903, 11777..."
